<a href="https://colab.research.google.com/github/allen6521/2023clup/blob/main/%E8%8F%9C%E5%96%AE%E6%8E%A8%E8%96%A6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 菜單推薦_數學版

##<準備>導入套件

1.導入pandas及numpy的套件

In [ ]:
import pandas as pd
import numpy as np
from pandas.core.frame import DataFrame
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

2.連結雲端硬碟

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##<一>將數據資料投入

1.讀取菜單數據

In [1]:
df=pd.read_excel("/content/drive/MyDrive/menu.xlsx")
df.to_csv("df_utf8.csv",encoding = "utf8")
df_csv=pd.read_csv("df_utf8.csv")
n=len(df_csv)
df_csv

NameError: name 'pd' is not defined

2.進行資料清洗

In [ ]:
df_csv.set_axis(["index","name",0,1,2,3],axis="columns",inplace=True)
ndf_csv=df_csv.drop(['index','name'],axis=1)
ndf_csv=ndf_csv.astype("float")
num_o=[]
for i in range(n):
  num_o.append(i+1)
num_i={"index":num_o}
num=DataFrame(num_i)
name=df_csv[['name']]
ndf_csv

,0,1,2,3
0,4.0,1.0,3.0,6.0
1,4.0,7.0,8.0,2.0
2,3.0,3.0,8.0,2.0
3,3.0,5.0,3.0,2.0
4,4.0,1.0,3.0,2.0
...,...,...,...,...
118,2.0,7.0,2.0,6.0
119,7.0,4.0,4.0,7.0
120,6.0,1.0,4.0,4.0
121,4.0,4.0,8.0,2.0


3.輸入客人選擇的菜色,根據選擇從菜單中擷取目標向量

In [ ]:
x="白片龍蝦"
goal=[]
for i in range(n):
  if x==name.iloc[i,0]:
    g_n=num.iloc[i,0]
    for k in range(4):
      goal.append(ndf_csv.iloc[i,k])
gl=pd.Series(goal)
gl=gl.astype("float")
gl

0    7.0
1    7.0
2    1.0
3    4.0
dtype: float64

##<二>計算所有菜色與客人菜色的內積，長度積

1.計算各菜色特質向量與目標向量內積



In [ ]:
r1=ndf_csv.dot(gl)
r1_f=pd.DataFrame(r1)
print(r1_f)

         0
0     62.0
1     93.0
2     58.0
3     67.0
4     46.0
..     ...
118   89.0
119  109.0
120   69.0
121   72.0
122   72.0

[123 rows x 1 columns]


2.計算各菜色特質向量與目標向量長度積

In [ ]:
ndf_arr=ndf_csv.values
l_ndf=np.linalg.norm(ndf_arr,axis=1,keepdims=True)
gl_arr=gl.values
l_gl=np.linalg.norm(gl_arr)
l_ndf_frame=pd.DataFrame(l_ndf)
l_frame=(l_gl*l_ndf_frame)
print(l_frame)

              0
0     84.439327
1    123.672956
2     99.448479
3     73.518705
4     58.736701
..          ...
118  103.416633
119  122.270193
120   89.078617
121  107.238053
122   91.624233

[123 rows x 1 columns]


3.將**內積**除以**長度積**,得到兩向量**夾角餘弦值**

In [ ]:
cos=[]
for i in range(n):
  cos.append(r1_f.iloc[i,0]/l_frame.iloc[i,0])
c_r=pd.DataFrame(cos)
print(c_r)

            0
0    0.734255
1    0.751983
2    0.583217
3    0.911333
4    0.783156
..        ...
118  0.860597
119  0.891468
120  0.774597
121  0.671403
122  0.785818

[123 rows x 1 columns]


4.將運算結果與原菜單進行合併

In [ ]:
c_n=pd.concat([num,name,c_r],axis=1)
c_n.rename(columns={0:"cos"},inplace=True)
c_n

,index,name,cos
0,1,螞蟻上樹,0.734255
1,2,蜜汁火腿,0.751983
2,3,八寶鴨,0.583217
3,4,棒棒雞,0.911333
4,5,荷葉粉蒸肉,0.783156
...,...,...,...
118,119,三不粘,0.860597
119,120,蚵爹,0.891468
120,121,老北京爆肚,0.774597
121,122,蠟味合蒸,0.671403


##<三>將夾角餘弦值進行排序

1.餘弦值越大，兩向量夾角越小，代表兩道菜特色越相近，因此新增排序函數，對餘弦值做降冪排序

In [ ]:
c_n_d=c_n.drop(index=[g_n-1])
c_n_d["rank"]=c_n_d["cos"].rank(method="dense",ascending=False)
c_n_d

,index,name,cos,rank
0,1,螞蟻上樹,0.734255,77.0
1,2,蜜汁火腿,0.751983,74.0
2,3,八寶鴨,0.583217,98.0
3,4,棒棒雞,0.911333,17.0
4,5,荷葉粉蒸肉,0.783156,58.0
...,...,...,...,...
118,119,三不粘,0.860597,35.0
119,120,蚵爹,0.891468,28.0
120,121,老北京爆肚,0.774597,62.0
121,122,蠟味合蒸,0.671403,90.0


2.依名次選擇特色最接近的5道菜

In [ ]:
rcm1=[df_csv.iloc[g_n-1,1]]
for l in range(6):
  for m in range(n-1):
    if c_n_d.iloc[m,3] == l:
      rcm1.append(c_n_d.iloc[m,1])
rcm_d1=[[rcm1[0],rcm1[1],rcm1[2],rcm1[3],rcm1[4],rcm1[5]]]
rcm2=["餘弦值"]
for l in range(6):
  for m in range(n-1):
    if c_n_d.iloc[m,3] == l:
      rcm2.append(c_n_d.iloc[m,2])
rcm_d2=[[rcm2[0],rcm2[1],rcm2[2],rcm2[3],rcm2[4],rcm2[5]]]
nu11=[["0","0","0","0","0","0"]]
rlt=pd.DataFrame(nu11)
rlt=rlt.append(rcm_d1,ignore_index=True)
rlt=rlt.append(rcm_d2,ignore_index=True)
menu_result1=rlt.drop([0])
menu_result1=menu_result1.rename({0:"選擇菜色",1:"推薦菜色1",2:"推薦菜色2",3:"推薦菜色3",4:"推薦菜色4",5:"推薦菜色5"},axis="columns")
menu_result1

,選擇菜色,推薦菜色1,推薦菜色2,推薦菜色3,推薦菜色4,推薦菜色5
1,白片龍蝦,碧螺蝦仁,松鼠黃魚,八寶蟳飯,鹽焗中蝦,魷魚螺肉蒜
2,餘弦值,0.987164,0.984313,0.983749,0.983749,0.983749


In [ ]:
menu_result1.to_excel('/content/drive/MyDrive/menu_result1.xlsx')

##<四>利用迴圈的方式,對所有菜色的推薦菜單進行演算

1.利用迴圈的方式,對所有菜色的推薦菜單進行演算

In [ ]:
n_l=[["0","0","0","0","0","0"]]
ndf=pd.DataFrame(n_l)
r=5
for x in range(1,n+1):
  ##根據選擇菜色擷取目標向量
  goal=[]
  for i in range(n):
    if x==num.iloc[i,0]:
      for j in range(4):
        goal.append(ndf_csv.iloc[i,j])
  gl=pd.Series(goal)
  gl=gl.astype("float")
  ##計算各菜色特質向量與目標向量內積
  r1=ndf_csv.dot(gl)
  r1_f=pd.DataFrame(r1)
  ##計算各菜色特質向量與目標向量長度積
  ndf_arr=ndf_csv.values
  l_ndf=np.linalg.norm(ndf_arr,axis=1,keepdims=True)
  gl_arr=gl.values
  l_gl=np.linalg.norm(gl_arr)
  l_ndf_frame=pd.DataFrame(l_ndf)
  l_frame=(l_gl*l_ndf_frame)
  ##計算各菜色特質向量與目標向量夾角
  cos=[]
  for k in range(n):
    cos.append(r1_f.iloc[k,0]/l_frame.iloc[k,0])
  c_r=pd.DataFrame(cos)
  ##合併為新Data
  c_n=pd.concat([num,name,c_r],axis=1)
  c_n.rename(columns={0:"cos"},inplace=True)
  ##進行排序
  c_n_d=c_n.drop(index=[x-1])
  c_n_d["r_d"]=c_n_d["cos"].rank(method="dense",ascending=False)
  rcm=[df_csv.iloc[x-1,1]]
  for l in range(r+1):
    for m in range(n-1):
      if c_n_d.iloc[m,3] == l:
        rcm.append(c_n_d.iloc[m,1])
  rcm_d=[[rcm[0],rcm[1],rcm[2],rcm[3],rcm[4],rcm[5]]]
  rcm_d
  ndf=ndf.append(rcm_d,ignore_index=True)

##<五>將演算結果輸出

1.列印推薦菜色

In [ ]:
ndf=ndf.drop([0])
menu_result2=ndf.rename({0:"選擇菜色",1:"推薦菜色1",2:"推薦菜色2",3:"推薦菜色3",4:"推薦菜色4",5:"推薦菜色5"},axis="columns")
menu_result2

,選擇菜色,推薦菜色1,推薦菜色2,推薦菜色3,推薦菜色4,推薦菜色5
1,螞蟻上樹,炸醬排骨,梅干扣肉,乾炸響鈴,無錫排骨,樟茶鴨
2,蜜汁火腿,紹興醉雞,花菇無黃蛋,貴妃雞,剁椒魚頭,蠟味合蒸
3,八寶鴨,蠟味合蒸,紅油炒手,百花鑲豆腐,麻辣黃瓜片,湯泡肚尖
4,棒棒雞,香菇雞湯,紅燒下巴,龍鬚燕丸,紅糟肉,罐煨佛跳牆
5,荷葉粉蒸肉,梅干扣肉,竹節鴿盅,老北京爆肚,辣椒豆古蒸魚,清蒸龍蝦
...,...,...,...,...,...,...
119,三不粘,油悶雙冬,髮絲牛百葉,叉燒肉,咕咾肉,光餅
120,蚵爹,白炒鮮竹蟶,腰果蝦仁,五更腸旺,西湖醋魚,荔浦扣肉
121,老北京爆肚,涮羊肉,龍井蝦仁,荷葉粉蒸肉,梅干扣肉,福州魚丸
122,蠟味合蒸,八寶鴨,紅油炒手,剁椒魚頭,蜜汁火腿,紹興醉雞


2.將結果存檔

In [ ]:
menu_result2.to_excel('/content/drive/MyDrive/menu_result2.xlsx')